In [79]:
# pip install --user kfp

In [80]:
from kfp import components
from kfp import dsl
import kfp

In [81]:
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]
PROJECT_ID

'hamoye-296618'

In [82]:
def preprocess_op(
    dataset="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
    image=f"gcr.io/{PROJECT_ID}/flowers-preprocess:latest",
    volume_mount=None
  
):
    
    return dsl.ContainerOp(
        name="flowers classifier preprocessor",
        image=image,
        command=["python", "preprocess.py"],
        arguments=[
            '--dataset', dataset,
        ],
        pvolumes={'/flowers': volume_mount.volume}
    )

In [83]:
def train_op(
    image=f"gcr.io/{PROJECT_ID}/flowers-train:latest",
    img_height=180,
    img_width=180,
    batch_size=32,
    volume_mount=None
):
    
    return dsl.ContainerOp(
        name="flowers classifier cnn model",
        image=image,
        command=["python", "train.py"],
        arguments=[
            '--batch-size', batch_size,
            '--img-height', img_height,
            '--img_width', img_width
        ],
        pvolumes={'/flowers': volume_mount.pvolume }
    )

In [84]:
@dsl.pipeline(
    name="Serving"
)
def cnn_flowers_pipeline(
    data_path='flowers/'

):
    volume = dsl.VolumeOp(
        name="image-volume",
        resource_name="flowers-volume",
        size="4Gi",
        modes=dsl.VOLUME_MODE_RWO
    )
    preprocess = preprocess_op(volume_mount=volume)
    train = train_op(volume_mount=preprocess)
    
    

In [1]:
import tensorflow as tf
tf.__version__

'2.4.0'

In [92]:
pip install --upgrade --user tensorflow

     |████████████████████████████████| 394.7 MB 2.9 kB/s eta 0:00:011                      | 28.4 MB 34.2 MB/s eta 0:00:1100:05237 kB/s eta 0:16:02��███████████▎                 | 175.5 MB 210 kB/s eta 0:17:22 kB/s eta 0:17:05     |████████████████                | 198.6 MB 210 kB/s eta 0:15:33| 233.1 MB 73.5 MB/s eta 0:00:031 MB 217 kB/s eta 0:10:27��████████████████████▋          | 266.5 MB 217 kB/s eta 0:09:49██████████████████          | 271.3 MB 147 kB/s eta 0:13:56��██████████████████▌     | 326.7 MB 56.1 MB/s eta 0:00:02�████████▋     | 327.9 MB 190 kB/s eta 0:05:509   | 331.7 MB 256 kB/s eta 0:04:06��████████████████████████▍   | 350.8 MB 56.5 MB/s eta 0:00:01     |████████████████████████████▋   | 352.7 MB 135 kB/s eta 0:05:11█████████████████████████████▍  | 362.0 MB 135 kB/s eta 0:04:02    |█████████████████████████████▌  | 363.6 MB 135 kB/s eta 0:03:51     |█████████████████████████████▊  | 367.2 MB 57.0 MB/s eta 0:00:01MB/s eta 0:00:01██████████████████████████████  | 369

Note: you may need to restart the kernel to use updated packages.


In [90]:
dir(tf.keras.preprocessing.image)

['DirectoryIterator',
 'ImageDataGenerator',
 'Iterator',
 'NumpyArrayIterator',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'apply_affine_transform',
 'apply_brightness_shift',
 'apply_channel_shift',
 'array_to_img',
 'img_to_array',
 'load_img',
 'random_brightness',
 'random_channel_shift',
 'random_rotation',
 'random_shear',
 'random_shift',
 'random_zoom',
 'save_img']

In [85]:
pipeline = kfp.Client().create_run_from_pipeline_func(cnn_flowers_pipeline, arguments={})